# Objective

Answer questions:  
* Given a current hand, what is the probability for competitors to declare Yaniv?  
* Given a current hand and being able to Yaniv, what is the probability of being Assafed?  

# Context and Definitions

What is the proability for each player to declare Yaniv?  

Definitions:  

Consistent throughout game:  
* `all_card` - all cards (depends if using Jokers or not)   
* $N_p^{total}$ - Number of players  

Varies throughout game:

* $N_i$ - number of cards held by competitor $i$  
* $N_p^{playing}$ - Number of players still in game  

Other variables:  
* $m_i$ - memory factor of compeitor $i$  
Is the fraction of unshown cards remembered by each player.  
This varies from $m_{min}$ (0-1)  to $m_{max}$ ($m_{min}$ to 1)
More advanced analysis might take into account: 
    * the time in which the card is no longer visible.  
    * difference if the card was discarded by user or competitor  
    * importance of card

* `seen` - set of cards already seen by the user  
This includes  
    * Current cards at hand  
    * Cards in pickup deck (set `pickup`)  
* `unseen` - set of cards not seen by the user.  
This includes  

    * `unknown` - set of available cards. 
    This includes: 
        * `available` - the cards available in the deck  
        * $s_i$ - set of cards held by competitor $i$ (`s_i`)  
      


Factors:  
* User's current hand  
These cards are excluded from `available`  
* Number of cards in competitor's hands  
* Cards seen picked up by others + memory factor  
* Cards discarded in pile + memory factor  
* Current card on top of pile  


Context:   
The game is focused around one users point-of-view (POV).  
The user knows about the content of the full set of cards (`all`), but knows about:  
* His current hand 
* Cards picked up by competitors (up to memory factor $m_i$)  
* Cards discarded by himself and competitors (up to memory factor $m_i$)

All the rest is missing information that must be inferred to deduce probabilities used for action.  

# Card Setup

In [1]:
from collections import OrderedDict

import numpy as np
import pandas as pd

In [2]:
jokers = True

suits = ['d', 'h', 'c', 's'] # diamonds, clubs, hearts, spades

values = range(1,11) + [10, 10, 10]
names = ['A'] + map(str, range(2,11)) + ['J', 'Q', 'K'] # Ace, 2-10, Jack, Queen, King

cards_all = {"{}{}".format(suit, name):
values[iname]  for iname, name in enumerate(names) for suit in suits}

if jokers:
    cards_all['joker1'] = 0 
    cards_all['joker2'] = 0 

cards_all = pd.Series(cards_all.values(), index=cards_all.keys())    
    
len(cards_all.keys())

54

In [33]:
cards_all


h8         8
h9         9
h2         2
h3         3
joker1     0
h6         6
h7         7
h4         4
h5         5
d8         8
d9         9
d6         6
d7         7
d4         4
d5         5
d2         2
d3         3
cK        10
cJ        10
s9         9
s8         8
d10       10
s3         3
s2         2
cA         1
joker2     0
s7         7
s6         6
s5         5
s4         4
cQ        10
c10       10
sQ        10
s10       10
sK        10
sJ        10
c9         9
c8         8
c3         3
c2         2
sA         1
c7         7
c6         6
c5         5
c4         4
dJ        10
dK        10
h10       10
hQ        10
dA         1
hJ        10
hK        10
hA         1
dQ        10
dtype: int64

# Game Simulation

class `game` 
Contains information of the game in general.  
* Players (total number, active number, names)   
* Keeping track of cards  
* Flag of joker usage


class `round`  
Contains information of a specific round    

class `player`  
Contains information of user:  
* In general  
* Spefic for round

In [70]:
l_names = ['John', 'Paul', 'Ringo', 'Jim']

class Game():
    def __init__(self, nplayers=3, jokers=True, thresh=200, verbose=True):
        # general definitions
        self.verbose = verbose
        
        if self.verbose:
            print "New Game"
        
        # may be varied by user
        self.nplayers = nplayers
        self.jokers = True
        self.thresh = thresh
        
        # determined by game rules
        self.round_number = 1
        self.nplayers_active = int(self.nplayers)
        
        self._player_setup()
        self._cards()
        
    def _player_setup(self):
        self.players = OrderedDict()
        for idx in range(self.nplayers):
            self.players[idx] = Player(idx)
    
    def _cards(self):
        # after a threshold of people we should have more than one 
        # standard deck. this would mean that full_deck indexes 
        # get a number to distinguish which deck each card came from
        suits = ['d', 'h', 'c', 's'] # diamonds, clubs, hearts, spades

        values = range(1,11) + [10, 10, 10]
        # Ace, 2-10, Jack, Queen, King
        names = ['A'] + map(str, range(2,11)) + ['J', 'Q', 'K'] 
        

        deck = {"{}{}".format(suit, name):
        values[iname]  for iname, name in enumerate(names) for suit in suits}

        if self.jokers:
            deck['joker1'] = 0 
            deck['joker2'] = 0 
            
        full_deck = pd.Series(deck.values(), index=deck.keys())  
            
        self.full_deck = full_deck
         
    def simulate_rounds(self, restart=True):
        
        if restart:
            self = Game(self.nplayers, jokers=self.jokers, verbose=self.verbose)
            # yields self.round_number = 1
        else:
            self.round_number += 1
        
        print 'here'
        self.round_ = Round(self.players)
        print 'now here'
        
    def hand2sum(hand):
        None
            
class Player():
    def __init__(self, idx):
        # constant in game
        self.idx = idx
        self.name = l_names[idx]
        
        # varies in game
        self.score = 0
        self.active = True  # all players are initially active
        
        # varies in round
        self.turn = False # True: turn to pick up or declare, False: Not turn 
        self.hand = None
        
    def hand_total():
        print None
    
    def check_active(thresh):
        # True: score<thresh, False: score >=tresh
        if self.score < thresh:
            self.active = True 
        else:
            self.active = False

class Round():
    def __init__(self, players, verbose=True):
        self.verbose = verbose
        
        self.players = players
        
        # counting the number of active players in round
        self.N_active = 0
        for idx in range(len(self.players)):
            if self.players[idx].active:
                self.N_active += 1
                
        if self.verbose:
            print "Number of players: {}".format(self.N_active)
            l_names = [player.name for idx, player in self.players.iteritems()]
            print ", ".join(l_names)
            
        self.distribute_cards()
        
    def distribute_cards(self):
        
        if self.verbose:
            print 'Distriubting cards'
            
        for idx, _ in self.players.iteritems():
            self.players[idx].hand = np.random.choice(game.full_deck.index.tolist(), 5, replace=False)
            print idx, self.players[idx].hand
        
            
game = Game()
game.simulate_rounds()

New Game
New Game
here
Number of players: 3
John, Paul, Ringo
Distriubting cards
0 ['hK' 'joker1' 'd8' 'c7' 's9']
1 ['d6' 'd2' 'dJ' 'd5' 'c2']
2 ['joker2' 's2' 'c8' 'sJ' 'hQ']
now here


In [73]:
Round([Player()])

TypeError: __init__() takes exactly 2 arguments (1 given)

In [67]:
aa

In [4]:
game.players[0].active

True

# Yaniv Declaring Probability

Given a stance in the game -  
what is the probability that each player can declare Yaniv?  

In [5]:
dict_N_maxValue = {} #given number of cards in hand, the maximum value card for Yaniv

# Evaluating the most extereme values to obtain a Yaniv
if jokers:
    dict_N_maxValue[5] = 5 # joker, joker, ace, ace, 5
    dict_N_maxValue[4] = 6 # joker, joker, ace, 6
    dict_N_maxValue[3] = 7 # joker, joker, 7
    dict_N_maxValue[2] = 7 # joker, 7
    dict_N_maxValue[1] = 7
else:
    dict_N_maxValue[5] = 3
    dict_N_maxValue[4] = 4
    dict_N_maxValue[3] = 5
    dict_N_maxValue[2] = 6
    dict_N_maxValue[1] = 7

In [6]:
# We will first attempt to answer this without knowing any cards of any of the players, 
# and at the beginning of the game when the pickup pile does not have cards


from itertools import combinations
from scipy.misc import comb

# ========= defintions =======
N_cards_i = 2
N_cards_possible_in_hand = len(cards_all.keys())
cards_available = cards_all.copy() # should also add cards known to be in possesion by player
# =============================

# this is combinations (not permutations, i.e [h1, d1] is the same as [d1, h1])
N_hands_possible = comb(N_cards_possible_in_hand, N_cards_i, exact=True)

max_value = dict_N_maxValue[N_cards_i]

# Determining the cards in range
cards_available_inRange = cards_available[cards_available <= max_value]

N_hands_yaniv = 0
for p in combinations(cards_available_inRange.values, N_cards_i):
    if np.sum(p) <= 7:
        N_hands_yaniv += 1
        
    # optimisation suggestion:
    # iterate over each index until the maximum combination obtain
    # and then calculate the number of optimsations
    # e.g, for N_i=5, once 1,1,1,1,3 is figured out, no need to check 1,1,1,2,3

print "Given a hand of {} cards\n{:,} total uknown cards\n{:,} of which in Yaniv range:".format(N_cards_i, len(cards_available), len(cards_available_inRange))
for key in sorted(cards_available_inRange.keys()): print key,
print "\nThere are a total of\n{:,} possible hands\n{:,} ({:0.6f}%) of which can call Yaniv ".format(N_hands_possible, N_hands_yaniv, N_hands_yaniv * 100. / N_hands_possible)

Given a hand of 2 cards
54 total uknown cards
30 of which in Yaniv range:
c2 c3 c4 c5 c6 c7 cA d2 d3 d4 d5 d6 d7 dA h2 h3 h4 h5 h6 h7 hA joker1 joker2 s2 s3 s4 s5 s6 s7 sA 
There are a total of
1,431 possible hands
219 (15.303983%) of which can call Yaniv 


In [7]:
# introducing known cards of user (POV)
# ========= defintions =======
N_cards_i = 5
# N_cards_possible_in_hand = len(cards_all.keys())
memory_i = 1.
# =============================

cards_user = cards_all.sample(N_cards_i, replace=False)
cards_user

# available cards
idx_cards_known = cards_user.index

# cards_available in user memory (for all cards memory_i=1)
cards_available = cards_all[~cards_all.index.isin(idx_cards_known)]

print "User cards: hand={} ".format(cards_user.values.sum()) 
for key in sorted(cards_user.keys()): print key,

User cards: hand=24 
c3 cA d6 h4 hJ


In [38]:
# Competitors' cards in user memory
# for now memory is the same.
# more realistic: user might remember cards differently for:
# * the player after him/her
# * the player with the least amount of cards in their hand
# * a player that picked up a desired card (e.g, low card, or one that the user needs for benefit)
# .... DEPENDS IF THE USER PICKS FROM THE SEEN-PICKUP or UNSEEN-PICKUP

N_players = 3
N_competitors = N_players - 1 

dict_players = {}

for idx in range(N_competitors):
    
    

In [35]:
# of Pile (current to pickup (might be more than one...) and all in user memory)

#pickup_all = pd.Series() # refresh in every game

pickup_current = cards_all.sample(3, replace=False)
#idx_cards_known = idx_cards_known.append(pickup_current.index)

# updating pickup pile
pickup_all = pickup_all.append(pickup_current)

# updating cards remembered by user
n_pickupMemory = int(len(pickup_all) * memory_i)
pickup_known = pickup_all.sample(n_pickupMemory, replace=False)

# verifying that current pickup is known
#if not pickup_current.index.isin(pickup_known.index):
#    pickup_known = pickup_known.append(pickup_known)
for idx in pickup_current.index.tolist():
    if idx not in pickup_known.index:
        pickup_known = pickup_known.append(pickup_current.loc[idx])
idx_cards_known = idx_cards_known.append(pickup_known.index)
    
cards_available = cards_all[~cards_all.index.isin(idx_cards_known)]
    
#cards_available = cards_all.copy() # should also add cards known to be in possesion by player
print "Current pickup card(s):", 
for key in sorted(pickup_current.keys()): print key,
print "\nKnown pickup cards:",
for key in sorted(pickup_known.keys()): print key,
print "\nThe total unaccounted for cards are: {}".format(len(cards_available))


Current pickup card(s): c2 s4 s9 
Known pickup cards: c2 c6 c7 h4 h8 hQ s4 s9 sQ 
The total unaccounted for cards are: 24


In [91]:
N_cards_possible_in_hand = len(cards_available.keys())

# this is combinations (not permutations, i.e [h1, d1] is the same as [d1, h1])
N_hands_possible = comb(N_cards_possible_in_hand, N_cards_i, exact=True)

max_value = dict_N_maxValue[N_cards_i] 

# Determining the cards in range
cards_available_inRange = cards_available[cards_available <= max_value]

N_hands_yaniv = 0
for p in combinations(cards_available_inRange.values, N_cards_i):
    if np.sum(p) <= 7:
        N_hands_yaniv += 1
        
    # optimisation suggestion:
    # iterate over each index until the maximum combination obtain
    # and then calculate the number of optimsations
    # e.g, for N_i=5, once 1,1,1,1,3 is figured out, no need to check 1,1,1,2,3

print "Given a hand of {} cards\n{:,} total uknown cards\n{:,} of which in Yaniv range:".format(N_cards_i, len(cards_available), len(cards_available_inRange))
for key in sorted(cards_available_inRange.keys()): print key,
print "\nThere are a total of\n{:,} possible hands\n{:,} ({:0.6f}%) of which can call Yaniv ".format(N_hands_possible, N_hands_yaniv, N_hands_yaniv * 100. / N_hands_possible)

Given a hand of 5 cards
48 total uknown cards
19 of which in Yaniv range:
c2 c3 c4 cA d2 d3 d4 d5 dA h2 h3 h4 h5 hA joker2 s2 s3 s4 s5 
There are a total of
1,712,304 possible hands
96 (0.005606%) of which can call Yaniv 


In [ ]:
# We now add one card to the pickup deck and see a user's hand

# Yaniv Winning Probability  
Given a stance in the game -  
* What is the probability of being Assafed by each player?  
* What is the probability of winning with a Yaniv?  

# Resources

[Python Combinations:   scipy.misc.comb](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.misc.comb.html)

[Python: Iterating over Permutation and Combinations](https://www.safaribooksonline.com/library/view/python-cookbook-3rd/9781449357337/ch04s09.html)

[Python HyperGeometric distributions:  `scipy.stats.hypergeom`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.hypergeom.html)

# Talk Abstract


Yaniv is a highly adictive card game which is very popular amongst backpackers. I discuss the statistical analysis of the game with a focus on python tools for probabilistic programming. 



https://en.wikipedia.org/wiki/Yaniv_(card_game)

Calling all Bayesian-minded analysts!

We will meet to work on projects themed around probabilistic programming.  
No matter your language of preference (be it is R, Stan or Python, e.g), bring your laptop and get ready for a few hours of discussing and applying statistical methods to questions that you are interested in answering. Come either with an idea that you have been itching to try out or willingness to work on somebody's else's. 

All levels welcome, no matter if you are a beginner data scientists, a developer with an interest in applying statistical methods or a statistician wanting to improve your applied skills. 

P(fun=True|attend=True) = 1 !

Disclaimer:
Frequentists welcome, too, but we'll set a flat prior on the conversion rate and update it with time.